In [ ]:
from aiy.vision.inference import CameraInference
from aiy.vision.models import face_detection
from aiy.vision.streaming.server import StreamingServer
from aiy.vision.streaming import svg
from aiy.leds import Leds, Color
from gpiozero import Servo
from aiy.pins import PIN_A

from picamera import PiCamera
from IPython.display import Image, display, clear_output

import contextlib
import time

### Joy Meter Demo

We are going to use the servo motor to indicate the level of joy detected. Feel free to tape a cardboard arrow to the servo arm or create a dail :) And remember if your joy detector is running, you need to turn it off using the commands

```
sudo systemctl stop joy_detection_demo.service
```

The code below initializes the servo assuming you connected it to PIN_A. Ignore the notics when runnig the code. Also it **seems the only way to run this cell twice is to restart the kernel** (still debugging), so run it once and then change and play wioth the code below.

In [ ]:
servo = Servo(PIN_A)

I added some code here that is called to create the overlay - basically the box around the face and the score above. You can customize it and/or add information you want to overlay on the camera feed. **This is not needed for the servo.**

In [ ]:
def svg_overlay(faces, frame_size):

    JOY_COLOR = (255, 70, 0)
    SAD_COLOR = (0, 0, 64)

    width, height = frame_size
    doc = svg.Svg(width=width, height=height)

    for face in faces:
        x, y, w, h = face.bounding_box
        fcol='rgb'+str(Color.blend(JOY_COLOR, SAD_COLOR, face.joy_score))
        doc.add(svg.Rect(x=int(x), y=int(y), width=int(w), height=int(h), rx=10, ry=10,
                         fill_opacity=0.3 * face.face_score,
                         style='fill:'+fcol+';stroke:white;stroke-width:4px'))

        doc.add(svg.Text('Joy: %.2f' % face.joy_score, x=x, y=y-10, fill='red', font_size=30))

    return str(doc)

#### Main loop

Here is our main loop based on the code we used last time. Look at the comments on what was changed. Basically we removed the part saving the picture and adjust the servo value to the joy score each frame. We also added the streaming back in, to while this cell runs, you can connect to http://orcspi-vis.local:4664 and see the stream.

In [ ]:
with contextlib.ExitStack() as stack:
    leds   = stack.enter_context(Leds())
    camera = stack.enter_context(PiCamera(sensor_mode=4, resolution=(820, 616)))

    # This starts and runs the streaming of the camera
    server = stack.enter_context(StreamingServer(camera))  

    print ("Loading model - hold on ..")
    
    # Move the servo to low (-1)
    servo.min()
    
    # Do inference on VisionBonnet
    with CameraInference(face_detection.model()) as inference:
        try:   
            for result in inference.run():
                leds.update(Leds.rgb_on(Color.RED))
                faces = face_detection.get_faces(result)
                
                # This sends the overlay (boxes) to add to the camera stream
                server.send_overlay(svg_overlay(faces, (result.width, result.height)))

                if len(faces) >= 1:
                    clear_output(wait=True)                 
                    leds.update(Leds.rgb_on(Color.GREEN))
                    print("Joy score: ", faces[0].joy_score)   
                    
                    # This sets the servo to the joy score value
                    # To use the full range (-1 to 1) we multiply the score by 2 and subtract 1
                    servo.value = 2*faces[0].joy_score  - 1
                                           
        except KeyboardInterrupt:
            print("Interrupted ..")
            
    leds.update(Leds.rgb_off())
    
    # Servo back to the middle upon ending
    servo.mid()
    
    print("Done")